In [1]:
import requests
import pandas as pd
import openpyxl

In [ ]:
offset_ = 0
objids = []

while objects_data.get('data').get('list')[0] != []:
    offset_ += 100
    limit_ = 100
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json()
    objects_data.get('data').get('list')[0]

    if objects_data.get('data').get('list')[0] == []:
        break

    objects_list = objects_data.get('data').get('list')
    objids.append([x.get('objId') for x in objects_list])

In [ ]:
objids[:10]

In [ ]:
all_res = []
for uid in objids:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{uid}'
    res = requests.get(url)
    res = res.json()
    all_res.append(res.get("data"))

all_res[0]

In [ ]:
houses = pd.DataFrame(data = pd.json_normalize(all_res))

In [ ]:
house_data = houses[[
            "region",
            "floorMax",
            "objElemLivingCnt",
            "objReady100PercDt",
            "objSquareLiving",
            "objElemParkingCnt",
            "objPriceAvg"
        ]]

In [ ]:
house_data.head()

In [ ]:
writer = pd.ExcelWriter('house_data.xlsx')
house_data.to_excel(writer)
writer.save()